In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
!pwd

/Users/paxton615/Github_Personal/Notes/Notes/Weather_stations


In [3]:
!ls

Download_station_data.ipynb Weather_stations
Untitled.ipynb


In [4]:
station_names = ['Aberporth', 'Armagh', 'Ballypatrick Forest', 'Bradford', 'Braemar', 'Camborne', 'Cambridge NIAB', 'Cardiff Bute Park',
 'Chivenor', 'Cwmystwyth', 'Dunstaffnage', 'Durham', 'Eastbourne', 'Eskdalemuir', 'Heathrow', 'Hurn', 'Lerwick',
 'Leuchars', 'Lowestoft', 'Manston', 'Nairn', 'Newton Rigg', 'Oxford', 'Paisley', 'Ringway', 'Ross-on-Wye',
 'Shawbury', 'Sheffield', 'Southampton', 'Stornoway Airport', 'Sutton Bonington', 'Tiree', 'Valley',
 'Waddington', 'Whitby', 'Wick Airport', 'Yeovilton']

In [5]:
lower_station_names = []
for i in station_names:
    lower_station_names.append(i.lower())

In [6]:
# lower_station_names

In [7]:
def data_clean(station_name):
    station_name = pd.read_excel('/Users/paxton615/Github_Personal/Notes/Notes/Weather_stations/Weather_stations/{}.xlsx'.format(station_name))
    
    station_name = station_name.iloc[1:,:-1] # get rid of useless row and column, 去除多余行，列
    col_names = ['year', 'month', 'tmax', 'tmin', 'af', 'rain', 'sun',
            'station', 'lat', 'lon'] # temperatre units: degC, rainfall units: mm, sun units: days
    station_name.columns = col_names # replace column names with col_names, 替换列名成 col_names
    
    # fill in station, lat and lon with its value, 给车站，经纬度填上唯一值
    station_name['station']= station_name.iloc[0, 7].lower().replace('-', '_').replace(' ','_')
    station_name['lat'] = station_name.iloc[0,8]
    station_name['lon'] = station_name.iloc[0,9]
    
    # replace --- with 0
    station_name = station_name.replace('---', 0)
    station_name = station_name.replace('--- #', 0)

    # fillna with 0s, 将所有NaN替换成 0
    station_name.fillna(0, inplace=True)
    
    # tidy all entries and unify types, 消除非数字字符，改type
    
    station_name['sun'] = station_name['sun'].astype(str)
    station_name['sun'] = station_name['sun'].str.extract(r'(\d+.\d+)').astype('float')

    station_name['tmax'] = station_name['tmax'].astype(str)
    station_name['tmax'] = station_name['tmax'].str.extract(r'(-?\d+.\d+)').astype('float')

    station_name['tmin'] = station_name['tmin'].astype(str)
    station_name['tmin'] = station_name['tmin'].str.extract(r'(-?\d+.\d+)').astype('float')

    station_name['rain'] = station_name['rain'].astype(str)
    station_name['rain'] = station_name['rain'].str.extract(r'(\d+.\d+)').astype('float')

    station_name['af'] = station_name['af'].astype(str)
    station_name['af'] = station_name['af'].str.extract(r'(\d+)').astype('float')

    
    # fillna with 0s, 将所有NaN替换成 0
    station_name = station_name.fillna(0)
    
    return station_name

In [8]:
aberporth = data_clean('Aberporth')

In [9]:
armagh = data_clean('Armagh')

In [10]:
ballypatrick_forest= data_clean('Ballypatrick Forest')

In [11]:
ballypatrick_forest

,year,month,tmax,tmin,af,rain,sun,station,lat,lon
1,1961.0,7.0,14.4,8.7,0.0,0.0,0.0,ballypatrick_forest,55.181,-6.153
2,1961.0,8.0,15.9,8.7,0.0,0.0,0.0,ballypatrick_forest,55.181,-6.153
3,1961.0,9.0,15.8,8.5,0.0,0.0,0.0,ballypatrick_forest,55.181,-6.153
4,1961.0,10.0,12.1,5.8,0.0,0.0,0.0,ballypatrick_forest,55.181,-6.153
5,1961.0,11.0,0.0,0.0,0.0,0.0,0.0,ballypatrick_forest,55.181,-6.153
...,...,...,...,...,...,...,...,...,...,...
725,2021.0,11.0,10.1,5.7,1.0,92.8,0.0,ballypatrick_forest,55.181,-6.153
726,2021.0,12.0,8.3,4.0,1.0,169.2,0.0,ballypatrick_forest,55.181,-6.153
727,2022.0,1.0,8.0,3.7,1.0,70.8,0.0,ballypatrick_forest,55.181,-6.153
728,2022.0,2.0,8.1,2.8,2.0,172.2,0.0,ballypatrick_forest,55.181,-6.153


In [12]:
ballypatrick_forest.af.unique()

array([ 0., 15.,  7.,  5., 19.,  3.,  2.,  6., 22., 20.,  4.,  8., 10.,
       13.,  9.,  1., 11., 18., 16., 12.])

In [38]:
ballypatrick_forest.tmin.unique()

array([ 8.7,  8.5,  5.8,  0. ,  0.5,  1.5, -1.5,  2. ,  4.6,  7.3,  8.8,
        7.8,  7. ,  3. , -1.4, -1.6,  2.1,  3.7,  9.1,  8.3,  3.9,  1.8,
        2.9,  4.2,  6.7,  9.6,  9.4,  9. ,  6. ,  0.8,  0.3,  1.2,  3.2,
        8.2,  8.4,  7.4,  1.9,  1.3,  1.4,  1.7,  2.7,  5.7,  9.5,  8.6,
        5.4,  2.5,  2.2,  2.4,  2.3,  3.1,  4.7,  9.3,  2.6, -1.1,  2.8,
        3.8,  7.9,  4. , -0.4,  5.3, 10.1, 10.7, -1. ,  0.7,  6.8, 10. ,
        5.6,  4.5,  5.9,  8.9,  6.9,  7.1,  6.2,  8.1, 10.3,  9.2,  6.4,
        3.6,  7.6, 10.6,  7.5,  3.3,  5.5, 10.8, -0.5, -0.2,  3.4,  1.6,
       -1.8,  0.2,  8. ,  0.9,  6.3,  9.7, 10.5, 10.2,  3.5,  9.9, -0.9,
        0.4,  5. ,  4.8, 10.9, 11.1, -0.6,  1. ,  1.1,  4.4,  9.8,  6.6,
       -0.1,  4.9,  0.6,  0.1,  7.7, 11.4,  4.3,  4.1, 11.3, 12. , 11.2,
       12.2,  6.1, 11. ,  7.2,  5.2, 12.3, 11.7, 11.9, 11.8,  6.5, -0.3,
       10.4,  5.1, 11.5])

In [39]:
ballypatrick_forest.dtypes

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [13]:
armagh.dtypes

year       float64
month      float64
tmax       float64
tmin       float64
af         float64
rain       float64
sun        float64
station     object
lat        float64
lon        float64
dtype: object

In [14]:
aberporth.dtypes

year       float64
month      float64
tmax       float64
tmin       float64
af         float64
rain       float64
sun        float64
station     object
lat        float64
lon        float64
dtype: object

In [15]:
aberporth = pd.read_excel('/Users/paxton615/Github_Personal/Notes/Notes/Weather_stations/Weather_stations/Wick Airport.xlsx')

In [16]:
col_names = ['year', 'month', 'tmax', 'tmin', 'af', 'rain', 'sun',
            'station', 'lat', 'lon']
# temperatre units: degC, rainfall units: mm, sun units: days

In [17]:
# get rid of useless row and column, 去除多余行，列
aberporth = aberporth.iloc[1:,:-1]
aberporth.head()

,yyyy,mm,tmax,tmin,af,rain,sun,Unnamed: 7,Unnamed: 8,Unnamed: 9
1,1914.0,1.0,5.9,0.9,---,48.3,---,Wick Airport,58.454,-3.088
2,1914.0,2.0,7.7,2.3,---,66,---,NaN,NaN,NaN
3,1914.0,3.0,6.4,1.1,---,76.7,---,NaN,NaN,NaN
4,1914.0,4.0,10.8,4.4,---,27.9,---,NaN,NaN,NaN
5,1914.0,5.0,10.7,4.4,---,61,---,NaN,NaN,NaN


In [18]:
# replace column names with col_names, 替换列名成 col_names
aberporth.columns = col_names
aberporth.head()

,year,month,tmax,tmin,af,rain,sun,station,lat,lon
1,1914.0,1.0,5.9,0.9,---,48.3,---,Wick Airport,58.454,-3.088
2,1914.0,2.0,7.7,2.3,---,66,---,NaN,NaN,NaN
3,1914.0,3.0,6.4,1.1,---,76.7,---,NaN,NaN,NaN
4,1914.0,4.0,10.8,4.4,---,27.9,---,NaN,NaN,NaN
5,1914.0,5.0,10.7,4.4,---,61,---,NaN,NaN,NaN


In [19]:
# fill in station, lat and lon with its value, 给车站，经纬度填上唯一值
aberporth['station']= aberporth.iloc[0, 7].lower().replace('-', '_').replace(' ','_')
aberporth['lat'] = aberporth.iloc[0,8]
aberporth['lon'] = aberporth.iloc[0,9]

In [20]:
aberporth

,year,month,tmax,tmin,af,rain,sun,station,lat,lon
1,1914.0,1.0,5.9,0.9,---,48.3,---,wick_airport,58.454,-3.088
2,1914.0,2.0,7.7,2.3,---,66,---,wick_airport,58.454,-3.088
3,1914.0,3.0,6.4,1.1,---,76.7,---,wick_airport,58.454,-3.088
4,1914.0,4.0,10.8,4.4,---,27.9,---,wick_airport,58.454,-3.088
5,1914.0,5.0,10.7,4.4,---,61,---,wick_airport,58.454,-3.088
...,...,...,...,...,...,...,...,...,...,...
1295,2021.0,11.0,9.3,4.4,2,109,---,wick_airport,58.454,-3.088
1296,2021.0,12.0,7.3,2.4,4,43.6,---,wick_airport,58.454,-3.088
1297,2022.0,1.0,8,2.9,3,49.4,---,wick_airport,58.454,-3.088
1298,2022.0,2.0,6.6,1,7,77.8,---,wick_airport,58.454,-3.088


In [21]:
aberporth.dtypes

year       float64
month      float64
tmax        object
tmin        object
af          object
rain        object
sun         object
station     object
lat        float64
lon        float64
dtype: object

In [22]:
aberporth.sun.unique()

array(['---', 88.1, 136.6, 165.3, 183.8, 108, 132.5, 131.7, 68.1, 41.2,
       23.1, 65.1, 68.2, 89.5, 151.8, 200.8, 183.4, 138, 197.8, 129.3,
       78.3, 33.1, 45.4, 46.1, 62.7, 127.6, 141, 176.8, 121.9, 195.1,
       248.4, 118.1, 74.7, 49.3, 28.7, 47.9, 79, 171.3, 163.6, 252.9,
       184.4, 147.7, 83.6, 133.2, 81.6, 59.3, 48.9, 39.4, 114.8, 85.5,
       131.9, 191.5, 244.5, 154.4, 117.5, 98.9, 96.4, 56.5, 34.7, 46.9,
       87.4, 108.9, 170.9, 198.1, 158.6, 163.3, 187.8, 117.6, 106.2, 67.3,
       24.2, 60.9, 56.2, 102.1, 161.4, 200, 180.5, 121.6, 145.7, 50, 51.8,
       63.9, 72.3, 122.7, 177.5, 115.2, 136, 166.1, 107.6, 94.3, 96.1,
       55.3, 29.2, 31.6, 84.1, 151.2, 163.9, 149.2, 136.9, 98.7, 155.4,
       115.5, 92.5, 55.8, 41.8, 29.3, 77.8, 102.6, 210.3, 138.2, 117.8,
       154.6, 115.4, 132, 92.8, 42, 21.5, 30.4, 84.5, 113.1, 177.1, 207.2,
       220.1, 200.6, 128.2, 133.6, 72, 47.3, 31.1, 37, 55.4, 96.6, 185.8,
       170.6, 170.4, 145.8, 97.3, 30, 22.4, 54, 83.3, 90, 18

In [23]:
# replace --- with 0
aberporth = aberporth.replace('---', 0)
aberporth = aberporth.replace('--- #', 0)

# fillna with 0s, 将所有NaN替换成 0
aberporth.fillna(0, inplace=True)

In [24]:
aberporth

,year,month,tmax,tmin,af,rain,sun,station,lat,lon
1,1914.0,1.0,5.9,0.9,0,48.3,0.0,wick_airport,58.454,-3.088
2,1914.0,2.0,7.7,2.3,0,66,0.0,wick_airport,58.454,-3.088
3,1914.0,3.0,6.4,1.1,0,76.7,0.0,wick_airport,58.454,-3.088
4,1914.0,4.0,10.8,4.4,0,27.9,0.0,wick_airport,58.454,-3.088
5,1914.0,5.0,10.7,4.4,0,61,0.0,wick_airport,58.454,-3.088
...,...,...,...,...,...,...,...,...,...,...
1295,2021.0,11.0,9.3,4.4,2,109,0.0,wick_airport,58.454,-3.088
1296,2021.0,12.0,7.3,2.4,4,43.6,0.0,wick_airport,58.454,-3.088
1297,2022.0,1.0,8,2.9,3,49.4,0.0,wick_airport,58.454,-3.088
1298,2022.0,2.0,6.6,1,7,77.8,0.0,wick_airport,58.454,-3.088


In [25]:
aberporth.dtypes

year       float64
month      float64
tmax        object
tmin        object
af          object
rain        object
sun        float64
station     object
lat        float64
lon        float64
dtype: object

In [26]:
aberporth['sun'] = aberporth['sun'].astype(str)
aberporth['sun'] = aberporth['sun'].str.extract(r'(\d+.\d+)').astype('float')

In [27]:
aberporth['tmax'] = aberporth['tmax'].astype(str)
aberporth['tmax'] = aberporth['tmax'].str.extract(r'(-?\d+.\d+)').astype('float')

In [28]:
aberporth['tmin'] = aberporth['tmin'].astype(str)
aberporth['tmin'] = aberporth['tmin'].str.extract(r'(-?\d+.\d+)').astype('float')

In [29]:
aberporth['rain'] = aberporth['rain'].astype(str)
aberporth['rain'] = aberporth['rain'].str.extract(r'(\d+.\d+)').astype('float')

In [30]:
aberporth['af'] = aberporth['af'].astype(str)
aberporth['af'] = aberporth['af'].str.extract(r'(\d+)').astype('float')

In [31]:
aberporth

,year,month,tmax,tmin,af,rain,sun,station,lat,lon
1,1914.0,1.0,5.9,0.9,0.0,48.3,0.0,wick_airport,58.454,-3.088
2,1914.0,2.0,7.7,2.3,0.0,NaN,0.0,wick_airport,58.454,-3.088
3,1914.0,3.0,6.4,1.1,0.0,76.7,0.0,wick_airport,58.454,-3.088
4,1914.0,4.0,10.8,4.4,0.0,27.9,0.0,wick_airport,58.454,-3.088
5,1914.0,5.0,10.7,4.4,0.0,NaN,0.0,wick_airport,58.454,-3.088
...,...,...,...,...,...,...,...,...,...,...
1295,2021.0,11.0,9.3,4.4,2.0,109.0,0.0,wick_airport,58.454,-3.088
1296,2021.0,12.0,7.3,2.4,4.0,43.6,0.0,wick_airport,58.454,-3.088
1297,2022.0,1.0,NaN,2.9,3.0,49.4,0.0,wick_airport,58.454,-3.088
1298,2022.0,2.0,6.6,NaN,7.0,77.8,0.0,wick_airport,58.454,-3.088


In [32]:
aberporth = aberporth.fillna(0)

In [33]:
aberporth

,year,month,tmax,tmin,af,rain,sun,station,lat,lon
1,1914.0,1.0,5.9,0.9,0.0,48.3,0.0,wick_airport,58.454,-3.088
2,1914.0,2.0,7.7,2.3,0.0,0.0,0.0,wick_airport,58.454,-3.088
3,1914.0,3.0,6.4,1.1,0.0,76.7,0.0,wick_airport,58.454,-3.088
4,1914.0,4.0,10.8,4.4,0.0,27.9,0.0,wick_airport,58.454,-3.088
5,1914.0,5.0,10.7,4.4,0.0,0.0,0.0,wick_airport,58.454,-3.088
...,...,...,...,...,...,...,...,...,...,...
1295,2021.0,11.0,9.3,4.4,2.0,109.0,0.0,wick_airport,58.454,-3.088
1296,2021.0,12.0,7.3,2.4,4.0,43.6,0.0,wick_airport,58.454,-3.088
1297,2022.0,1.0,0.0,2.9,3.0,49.4,0.0,wick_airport,58.454,-3.088
1298,2022.0,2.0,6.6,0.0,7.0,77.8,0.0,wick_airport,58.454,-3.088


In [34]:
aberporth.dtypes

year       float64
month      float64
tmax       float64
tmin       float64
af         float64
rain       float64
sun        float64
station     object
lat        float64
lon        float64
dtype: object

In [35]:
aberporth.af.unique()

array([ 0.,  3.,  9., 14.,  5.,  8.,  1., 10.,  2.,  6.,  4.,  7., 13.,
       15., 16., 12., 23., 24., 17., 11., 19., 21., 22., 25., 20.])

In [36]:
aberporth.tmax.unique()

array([ 5.9,  7.7,  6.4, 10.8, 10.7, 13.5, 14.4, 15.2, 13.4, 11.8,  8.8,
        5.7,  5.6,  5.5,  6.7,  8.7,  9.4, 12.6, 14.3, 14.6, 13.2, 10.9,
        6.3,  5.4,  5.3,  4.7,  9.7,  0. , 13.8, 11.1,  4.6,  6.8, 10.1,
       14.9,  9.2,  4.3,  7.2,  7.3,  7.4, 11.7, 13.6, 11.5,  8.5,  6.9,
        5.8,  4.9, 13.7, 14.7, 13.3, 10.2,  7.8, 14.8,  9.9,  7.9,  9.6,
       11.9, 14.1, 12.2,  7.1, 12.5,  8.1,  9.3, 12.8,  6.1,  8.2, 12.1,
        6.6, 10.4, 15.4,  4.1,  7.6,  9.8, 15.1,  6.2,  8.9, 11.3, 11.2,
       13.1,  8.6,  4.8,  9.1,  6.5,  5.2,  8.3, 10.6, 14.2, 15.5, 15.6,
        8.4, 10.3, 17.3, 11.4, 15.8, 12.9, 16.3, 16.1,  7.5, 14.5, 10.5,
        5.1,  3.9,  4.5,  9.5, 15.9, 16.8,  3.7, 16.7, 16.6,  2.2, 18.3,
       11.6, 15.3,  3.8, 15.7, 12.3, 13.9, 16.4, 12.4,  3.1, 18.1, 12.7,
        3.6, 17.1,  3.2,  4.2, 17.4, 16.2, 17.6, 17.8,  3.4,  4.4, 16.9,
       18.4, 16.5, 18.5,  3.5, 18.2, 17.2, 18.6])

In [37]:
aberporth.tmin.unique()


array([ 0.9,  2.3,  1.1,  4.4,  7.6, 10.2,  7.9,  6.1,  0. ,  0.5,  1.3,
        2.4,  3.6,  6.8,  8.9, 10.3,  8.7,  5.2,  1.2,  0.2,  2.8,  0.6,
        5.1,  7.3,  9.6,  8.8,  8.1,  5.4,  4.3,  0.8,  0.4,  4.7,  7.5,
        9.5, 10.7,  8.3,  3.1, -1.8,  1.8,  6.9,  6.2,  9.7,  5.3,  5.6,
        2.7, -1.1, -0.4,  6.3,  7.7,  6.7,  1.7,  1.9,  2.9,  8.6,  8.4,
        7.1,  1.6,  3.7,  9.3,  8.5,  7.4,  3.4,  3.3,  1.4,  7.8,  7.2,
        5.9,  3.8,  3.9, 10.1,  2.6,  6.5, 10.8,  2.5,  8.2,  2.1,  5.8,
        2.2,  4.2,  9.4,  6.6,  4.5,  3.5,  9.2,  0.7,  9.9,  9.8, -0.5,
        4.8,  5.5, 10.4,  4.6,  5.7, 11.5, 10.5,  3.2, -0.2,  6.4, 10.6,
       -0.6,  0.3,  1.5, -2.5,  4.1, 10.9, -2.9, -0.1, -1.6, -2.4, 11.1,
        9.1, -0.8, -0.9, -2.3, -1.3, -0.3,  0.1, -2.1, -0.7, -1.7, -1.9,
       -1.4,  4.9, 11.8, 11.4, 11.2, 11.3])